In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold


import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

### Holdout sampling (not used)

In [ ]:
dataset = pd.read_csv("training_dataset.csv").drop("Unnamed: 0", axis = 1)
test_dataset = pd.read_csv("test_dataset.csv").drop("Unnamed: 0", axis = 1)

#dataset

In [ ]:
# mapeamento do atributo alvo (weather)
key_mapping = {value : i for i, value in enumerate(pd.unique(dataset['weather']))}
reverse_key_mapping = {value : key for key, value in key_mapping.items()}

#key_mapping

#### Preprocessing (holdout)

In [ ]:
# mapeamento do atributo alvo (weather)
key_mapping = {value : i for i, value in enumerate(pd.unique(dataset['weather']))}
reverse_key_mapping = {value : key for key, value in key_mapping.items()}

key_mapping




In [ ]:
dataset['weather'] = dataset['weather'].map(key_mapping)
#dataset

In [ ]:
#padroniza os dados do dataset 
for variavel_climatica in ['humidity', 'pressure', 'temperature', 'wind']:
    scale = StandardScaler().fit(dataset[variavel_climatica].to_numpy().reshape(-1, 1))  #cria a escala para cada dado do dataset
    dataset[variavel_climatica] = scale.transform(dataset[variavel_climatica].to_numpy().reshape(-1, 1))  #transforma os dados do dataset

#dataset


In [ ]:
#dataset.describe()

In [ ]:
#converte de type: Dataset para type: np.array
y = to_categorical(dataset['weather'].to_numpy(), num_classes=len(pd.unique(dataset['weather'])))   # one-hot encoding baseado na quantidade de weathers 
                                                                                                    # categorical crossentropy usa one-hot, sparse categorical crossentropy usa inteiros 
x = np.column_stack([dataset['humidity'].to_numpy().astype(np.float32)     
     , dataset['pressure'].to_numpy().astype(np.float32)
     , dataset['temperature'].to_numpy().astype(np.float32)
     , dataset['wind'].to_numpy().astype(np.float32)])


In [ ]:
#as classes são desbalanceadas, é necessário balancear elas
#dataset['weather'].value_counts()



In [ ]:
# balancea as classes (uma vez que a amostragem é desproporcional), dado pesos para cada weather durante o treinamento do modelo
weights = {key:weight for key, weight in enumerate(class_weight.compute_class_weight('balanced',  classes = pd.unique(dataset['weather']), y=  dataset['weather']))}
#weights

#### Architecture definition (holdout)

In [ ]:
def architecture():
    neural_network = tf.keras.models.Sequential([
                    #layers.Dense(1024, activation= 'relu'), #ignore, Optional:  input_dim = 4,
                    #layers.Dense(2048, activation= 'relu'), 
                    #layers.Dense(32768, activation= 'relu'),
                    #layers.Dense(16384, activation= 'relu'),
                    #layers.Dense(4096, activation= 'relu'),
                    layers.Dense(2048, activation= 'relu'),
                    #tf.keras.layers.Dropout(rate = 0.5),
                    layers.Dense(1024, activation= 'relu'),
                    #tf.keras.layers.Dropout(rate = 0.4),
                    layers.Dense(512, activation= 'relu'),
                    #tf.keras.layers.Dropout(rate = 0.3),
                    layers.Dense(256, activation= 'relu'),
                    #tf.keras.layers.Dropout(rate = 0.2),
                    #layers.Dense(128, activation= 'relu'),
                    #layers.Dense(128, activation= 'relu'),
                    #layers.Dense(64, activation= 'relu'),
                    layers.Dense(len(pd.unique(dataset['weather'])), activation= 'softmax')])
    
    return neural_network

In [ ]:
class callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs=None):  # altera o comportamento do metodo herdado de Callback
    global VALUE_CHECKPOINT
    global fold
    
    #metrics = list(logs.values())

    checkpoint_path = f'checkpoint({fold},{VALUE_CHECKPOINT}_loss)'

    if logs.get('loss') < VALUE_CHECKPOINT:       #verifica se a loss atinge valores menores que o especificado
      self.model.save_weights(checkpoint_path, save_format='tf')
      self.model.stop_training = True
        

#### Training (holdout)

In [ ]:
neural_network = architecture()

In [ ]:
optimizer = Adam(learning_rate=0.0001, clipvalue=0.5)

neural_network.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [ ]:
class callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs=None):  # altera o comportamento do metodo herdado de Callback
    global VALUE_CHECKPOINT
    
    #metrics = list(logs.values())

    checkpoint_path = f'checkpoint({VALUE_CHECKPOINT}_loss)'

    if logs.get('loss') < VALUE_CHECKPOINT:       #verifica se a loss atinge valores menores que 1.8
      self.model.save_weights(checkpoint_path, save_format='tf')
      self.model.stop_training = True
        

In [ ]:
#neural_network.build(input_shape = (4,1))
neural_network.load_weights('checkpoint')


In [ ]:
VALUE_CHECKPOINT = 0.26
epochs = 1000

# Treinamento do modelo
report_metrics = neural_network.fit(x, y, epochs=epochs, callbacks=callback(), class_weight=weights)

# Gravação dos valores de loss no arquivo report.txt
with open(f'report({VALUE_CHECKPOINT}_loss).txt', 'w') as report:
    report.write(','.join(map(str, range(1, epochs + 1))) + '\n')  # Escreve os números de épocas no cabeçalho
    for loss in report_metrics.history['loss']:
        report.write(f'{loss},')




In [ ]:
neural_network.fit(x,y, epochs = 3, callbacks = callback(), class_weight= weights)

### K fold sampling

In [2]:
dataset = pd.read_csv("dados_originais.csv").drop("Unnamed: 0", axis = 1).rename({'month': 'season'},axis = 1)
dataset

,season,humidity,pressure,temperature,wind,weather
0,spring,58,1012,15,7,few_clouds
1,spring,57,1012,15,7,few_clouds
2,spring,57,1012,15,7,few_clouds
3,spring,57,1012,15,7,few_clouds
4,spring,57,1012,15,6,few_clouds
...,...,...,...,...,...,...
43575,spring,36,1019,16,3,sky_is_clear
43576,spring,38,1019,16,1,sky_is_clear
43577,spring,54,1019,14,2,sky_is_clear
43578,spring,62,1020,12,3,sky_is_clear


#### Preprocessing (k fold)

In [3]:
# mapeamento do atributo alvo (weather)
key_mapping = {value : i for i, value in enumerate(pd.unique(dataset['weather']))}
reverse_key_mapping = {value : key for key, value in key_mapping.items()}

key_mapping

{'few_clouds': 0,
 'scattered_clouds': 1,
 'broken_clouds': 2,
 'sky_is_clear': 3,
 'overcast_clouds': 4,
 'mist': 5,
 'drizzle': 6,
 'moderate_rain': 7,
 'light_intensity_drizzle': 8,
 'light_rain': 9,
 'fog': 10,
 'haze': 11,
 'heavy_snow': 12,
 'heavy_intensity_drizzle': 13,
 'heavy_intensity_rain': 14,
 'light_rain_and_snow': 15,
 'snow': 16,
 'light_snow': 17,
 'proximity_thunderstorm': 18,
 'thunderstorm': 19,
 'thunderstorm_with_rain': 20,
 'thunderstorm_with_heavy_rain': 21,
 'thunderstorm_with_light_rain': 22,
 'very_heavy_rain': 23,
 'dust': 24}

In [4]:
#Faz o mapeamento do atributo season em X
key_mapping_season = {value : round(i/3, 3) for i, value in enumerate(pd.unique(dataset['season']))}  
key_mapping_season

{'spring': 0.0, 'summer': 0.333, 'autumn': 0.667, 'winter': 1.0}

In [5]:
dataset['season'] = dataset['season'].map(key_mapping_season)
dataset

,season,humidity,pressure,temperature,wind,weather
0,0.0,58,1012,15,7,few_clouds
1,0.0,57,1012,15,7,few_clouds
2,0.0,57,1012,15,7,few_clouds
3,0.0,57,1012,15,7,few_clouds
4,0.0,57,1012,15,6,few_clouds
...,...,...,...,...,...,...
43575,0.0,36,1019,16,3,sky_is_clear
43576,0.0,38,1019,16,1,sky_is_clear
43577,0.0,54,1019,14,2,sky_is_clear
43578,0.0,62,1020,12,3,sky_is_clear


In [6]:
# cria um dicionario com a chave sendo cada k_fold e os values são outro dicionario com cada key sendo o tipo de objeto (treinamento ou validacao) e os valores são um dataframe
k_folds = { fold : {"x_train": pd.read_csv(f'./K Folds Cross Validation/{fold}/x_train_{fold}').drop("Unnamed: 0", axis = 1),
                    "x_val": pd.read_csv(f'./K Folds Cross Validation/{fold}/x_val_{fold}').drop("Unnamed: 0", axis = 1),
                    "y_train": pd.read_csv(f'./K Folds Cross Validation/{fold}/y_train_{fold}').drop("Unnamed: 0", axis = 1)['weather'],  #abre o dataframe e ja faz o mapeamento
                    "y_val": pd.read_csv(f'./K Folds Cross Validation/{fold}/y_val_{fold}').drop("Unnamed: 0", axis = 1)['weather'],}     #abre o dataframe e ja faz o mapeamento
                     
                     for fold in os.listdir("./K Folds Cross Validation/")}
    
k_folds


{'fold_0': {'x_train':        season  humidity  pressure  temperature  wind
  0         0.0        69      1015            4     3
  1         0.0        51      1022            0     5
  2         0.0        51      1022           -1     4
  3         0.0        55      1022           -1     4
  4         0.0        51      1022           -1     5
  ...       ...       ...       ...          ...   ...
  34859     0.0        36      1019           16     3
  34860     0.0        38      1019           16     1
  34861     0.0        54      1019           14     2
  34862     0.0        62      1020           12     3
  34863     0.0        58      1020           11     2
  
  [34864 rows x 5 columns],
  'x_val':       season  humidity  pressure  temperature  wind
  0      0.000        58      1012           15     7
  1      0.000        57      1012           15     7
  2      0.000        57      1012           15     7
  3      0.000        57      1012           15     7
  4      

In [10]:
# #mapeia as estacoes do ano (season) para float
# for fold, data in k_folds.items():
#     data['x_train']['season'] = data['x_train']['season'].map(key_mapping_season)
#     data['x_val']['season'] = data['x_val']['season'].map(key_mapping_season)

# np.unique(k_folds[fold][data]['season'])

In [11]:
#np.unique(k_folds['fold1']['x_train']['season'])


In [ ]:
#padroniza os dados do dataset 
scaler = StandardScaler()

for fold, data in k_folds.items():
    x_train = data["x_train"]
    x_val = data["x_val"]

    # Ajusta e transforma o conjunto de treinamento
    x_train = scaler.fit_transform(x_train)
    
    # Ajusta e transforma o conjunto de validação
    x_val = scaler.transform(x_val)

    # Atualize os DataFrames no dicionário
    k_folds[fold]["x_train"] = x_train
    k_folds[fold]["x_val"] = x_val



In [13]:
# define os pesos para cada fold 
weights = {}

for fold, data in k_folds.items():
    y_train = data["y_train"]

    weights[fold] = {key: weight for key, weight in enumerate(class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train))}

weights




{'fold_0': {0: 0.533700727133563,
  1: 0.38867335562987737,
  2: 0.2837932437932438,
  3: 0.15370439766339689,
  4: 0.40398609501738125,
  5: 0.32281481481481483,
  6: 7.878870056497175,
  7: 1.8668808567603747,
  8: 4.358,
  9: 0.4415959468017733,
  10: 1.6178190255220417,
  11: 2.526376811594203,
  12: 14.526666666666667,
  13: 87.16,
  14: 4.7273220338983055,
  15: 154.95111111111112,
  16: 17.878974358974357,
  17: 4.557385620915032,
  18: 15.158260869565217,
  19: 23.636610169491526,
  20: 87.16,
  21: 154.95111111111112,
  22: 48.08827586206897,
  23: 21.12969696969697,
  24: 82.03294117647059},
 'fold_1': {0: 0.533700727133563,
  1: 0.38867335562987737,
  2: 0.2837932437932438,
  3: 0.15370439766339689,
  4: 0.4038690993339125,
  5: 0.3228895577680019,
  6: 7.878870056497175,
  7: 1.8668808567603747,
  8: 4.344423676012461,
  9: 0.4417358251504593,
  10: 1.6178190255220417,
  11: 2.526376811594203,
  12: 14.526666666666667,
  13: 87.16,
  14: 4.7273220338983055,
  15: 154.951111

#### Architecture (k fold)

In [14]:
def architecture():
    neural_network = tf.keras.models.Sequential([
    
                    layers.Dense(2048, activation= 'relu'),
                    tf.keras.layers.Dropout(rate = 0.3),
                    layers.Dense(1024, activation= 'relu'),
                    tf.keras.layers.Dropout(rate = 0.2),
                    layers.Dense(512, activation= 'relu'),
                    tf.keras.layers.Dropout(rate = 0.1),
                    layers.Dense(256, activation= 'relu'),

                    layers.Dense(len(key_mapping.keys()), activation= 'softmax')]) # ignore: pd.unique(dataset['weather']))
    return neural_network



In [15]:
class callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs=None):  # altera o comportamento do metodo herdado de Callback
    global VALUE_CHECKPOINT
    global fold
    
    #metrics = list(logs.values())

    checkpoint_path = f'checkpoint({fold},{VALUE_CHECKPOINT}_loss)'

    if logs.get('loss') < VALUE_CHECKPOINT:       #verifica se a loss atinge valores menores que o especificado
      self.model.save_weights(checkpoint_path, save_format='tf')
      self.model.stop_training = True

#### Training (k fold)

In [16]:
neural_network = architecture()

In [17]:
optimizer = Adam(learning_rate=0.00001, clipvalue=0.5)

neural_network.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [18]:
neural_network.load_weights('checkpoint')

In [20]:
#treina e valida com k-folds cross-validation
epochs = 300
VALUE_CHECKPOINT = 1.5

for fold in k_folds.keys():
    x = k_folds.get(fold).get("x_train")
    y = k_folds.get(fold).get("y_train")
    x_val = k_folds.get(fold).get("x_val")
    y_val = k_folds.get(fold).get("y_val")
    
    y = to_categorical(np.array(y), num_classes=len(np.unique(y)))   # one-hot encoding baseado na quantidade de classes 
                                                                     # categorical crossentropy usa one-hot, sparse categorical crossentropy usa inteiros 
    #x = np.column_stack([x['humidity'], x['pressure'], x['temperature'], x['wind']])

    y_val = to_categorical(np.array(y_val), num_classes=len(np.unique(y_val)))

    #x_val = np.column_stack([x_val['humidity'], x_val['pressure'], x_val['temperature'], x_val['wind']])
    
    #carrega o checkpoint
    #neural_network.load_weights('checkpoint')
    
    # Treinamento do modelo
    report_metrics = neural_network.fit(x, y, validation_data = (x_val, y_val), epochs = epochs, callbacks = callback(), class_weight= weights[fold]) 
    
    # Gravação dos valores de loss no arquivo report.txt
    with open(f'report({fold}).txt', 'w') as report:
        report.write(','.join(map(str, range(1, epochs + 1))) + '\n')  # Escreve os números de épocas no cabeçalho
        for loss in report_metrics.history['loss']:
            report.write(f'{loss},')
            
        report.write(','.join(map(str, range(1, epochs + 1))) + '\n')
        for val_loss in report_metrics.history['val_loss']:
            report.write(f'{val_loss},')


273/273 [==============================] - 1s 4ms/step


NameError: name 'report_metrics' is not defined

#### Visualization (k fold)

In [2]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
ROOT_PATH = r"C:\Users\leand\OneDrive\Área de Trabalho\Aprendizado de Maquina\Neural Network\report (k fold cross validation)"

vis_fold0 = pd.read_csv(fr"{ROOT_PATH}\report(fold_0).txt").transpose().dropna().rename({0:'fold0_loss', 1:'fold0_val_loss'},axis =1)
vis_fold1 = pd.read_csv(fr"{ROOT_PATH}\report(fold_1).txt").transpose().dropna().rename({0:'fold1_loss', 1:'fold1_val_loss'},axis =1)
vis_fold2 = pd.read_csv(fr"{ROOT_PATH}\report(fold_2).txt").transpose().dropna().rename({0:'fold2_loss', 1:'fold2_val_loss'},axis =1)
vis_fold3 = pd.read_csv(fr"{ROOT_PATH}\report(fold_3).txt").transpose().dropna().rename({0:'fold3_loss', 1:'fold3_val_loss'},axis =1)
vis_fold4 = pd.read_csv(fr"{ROOT_PATH}\report(fold_4).txt").transpose().dropna().rename({0:'fold4_loss', 1:'fold4_val_loss'},axis =1)

In [4]:
vis_fold4

,fold4_loss,fold4_val_loss
1,1.703093,2.313956
2,1.616244,2.356901
3,1.577400,2.391842
4,1.558566,2.437394
5,1.552440,2.480854
6,1.542642,2.445291
7,1.529354,2.449275
8,1.534130,2.487574
9,1.514807,2.443268
10,1.510799,2.460400


In [5]:
y = pd.concat([vis_fold0.transpose(), vis_fold1.transpose(), vis_fold2.transpose(), vis_fold3.transpose(), vis_fold4.transpose()], axis = 0)
y

,1,2,3,4,5,6,7,8,9,10,...,75,76,77,78,79,80,81,82,83,84
fold0_loss,2.185244,2.094740,2.035345,2.006847,1.946261,1.933436,1.890831,1.860954,1.832695,1.832894,...,1.516016,1.514842,1.511528,1.522926,1.514695,1.510809,1.508050,1.502052,1.502463,1.496677
fold0_val_loss,2.474042,2.531891,2.512506,2.532838,2.545415,2.639750,2.566860,2.561625,2.627091,2.658193,...,2.766835,2.724256,2.806243,2.755749,2.693523,2.678050,2.787148,2.820018,2.781705,2.840706
fold1_loss,2.096956,1.950519,1.884071,1.834502,1.794544,1.771286,1.730780,1.700978,1.691618,1.676683,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fold1_val_loss,2.416390,2.495011,2.527581,2.468618,2.455825,2.551908,2.465409,2.595078,2.583162,2.526187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fold2_loss,2.007475,1.880930,1.804555,1.765062,1.732552,1.725136,1.696676,1.695072,1.672331,1.667623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fold2_val_loss,2.450819,2.499288,2.577780,2.482836,2.456696,2.435712,2.472884,2.536083,2.578144,2.490822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fold3_loss,1.786784,1.683282,1.660466,1.635120,1.631279,1.598944,1.601023,1.579599,1.569772,1.579088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fold3_val_loss,2.416028,2.337203,2.477544,2.408122,2.475358,2.451550,2.371228,2.400359,2.462310,2.353626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fold4_loss,1.703093,1.616244,1.577400,1.558566,1.552440,1.542642,1.529354,1.534130,1.514807,1.510799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fold4_val_loss,2.313956,2.356901,2.391842,2.437394,2.480854,2.445291,2.449275,2.487574,2.443268,2.460400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
fig = px.line(y.transpose(), template = 'plotly_white', title= 'Loss x Epoch', width=1200, height=600,
        color_discrete_sequence=['Blue', 'CornflowerBlue', 'Red', 'IndianRed', 'Green', 'LightGreen', 'Purple', 'Plum', 'Yellow', 'Khaki' ])     #'LightBlue','LightPink','dodgerblue','hotpink'
fig['layout']['xaxis']['title']['text'] = 'Epoch'
fig['layout']['yaxis']['title']['text'] = 'Loss '  


fig.update_layout(font = dict(size=17))#,xaxis_range=[24,35])
fig.update_xaxes(dtick = 1)

#fig.update_traces(y = y.iloc[1],marker= dict(symbol = ('3'),size = 5,color = 'black'))
#fig.update_traces(y = y.iloc[2],marker= dict(symbol = ('4'),size = 5,color = 'red'))
fig.show()            

### Evaluation

In [ ]:
#padroniza os dados do dataset de teste
for variavel_climatica in ['humidity', 'pressure', 'temperature', 'wind']:
    scale = StandardScaler().fit(test_dataset[variavel_climatica].to_numpy().reshape(-1, 1))  #cria a escala para cada dado do dataset
    test_dataset[variavel_climatica] = scale.transform(test_dataset[variavel_climatica].to_numpy().reshape(-1, 1))  #transforma os dados do dataset

test_dataset

In [ ]:
test_dataset[test_dataset['weather'] == 'sky_is_clear']

In [ ]:
#testa uma amostra xi
xi = [0.373659,	-2.581229,	-0.317683,	-0.104134] # objeto teste
prediction = {key: round(probability*100, 3) for key , probability in enumerate(neural_network.predict(np.array(xi).reshape(1,-1))[0])}

for key, value in key_mapping.items():
    prediction[key] = prediction.pop(value)

prediction #retorna a probabilidade de ocorrencia para cada classe 


In [ ]:
#calcula a matriz de confusão 
count = 0
for i in test_dataset.index: #testa cada amostra contida no dataset de teste
    xi = [test_dataset.loc[i, 'humidity'], test_dataset.loc[i, 'pressure'], test_dataset.loc[i, 'temperature'], test_dataset.loc[i, 'wind']]
    current_prediction = {key: round(probability*100, 3) for key , probability in enumerate(neural_network.predict(np.array(xi).reshape(1,-1), verbose = 0)[0])}

    #verifica a maior probabilidade calculada pelo modelo e a sua classe
    network_choice = [0,0,0,0,]
    classe1 = 0
    classe2 = 0
    classe3 = 0
    classe4 = 0


    #contagem de acertos do modelo (dada a complexidade do dataset, considera um acerto caso o groundtruth esteja entre as 3 maiores probabilidades)
    values_mapping = {probability : weather for weather, probability in current_prediction.items()} # salva os valores a predicao atual (para que as probabilidades sejam organizadas em ordem crescente para a comparação)

    for probability in sorted(current_prediction.values()):
        if probability > network_choice[0]:
            classe1, classe2, classe3, classe4 = values_mapping.get(probability), classe1, classe2, classe3
            network_choice[0], network_choice[1], network_choice[2], network_choice[3] = probability, network_choice[0], network_choice[1],  network_choice[2]

    # #contagem de acertos do modelo (considera um acerto caso o groundtruth esteja entre as 3 maiores probabilidades)
    # for key in current_prediction.keys():
    #     if current_prediction.get(key) > network_choice[0]:
    #         classe1, classe2, classe3, classe4 = key, classe1, classe2, classe3
    #         network_choice[0], network_choice[1], network_choice[2], network_choice[3] = current_prediction.get(key), network_choice[0], network_choice[1],  network_choice[2]
    
    
    #print(reverse_key_mapping.get(classe), test_dataset.loc[i, 'weather'])
    if reverse_key_mapping.get(classe1) == test_dataset.loc[i, 'weather'] or reverse_key_mapping.get(classe2) == test_dataset.loc[i, 'weather'] or reverse_key_mapping.get(classe3) == test_dataset.loc[i, 'weather']:
        print('SIM \n\n\n')
        count+=1
        print(count,i)

    

In [ ]:
reverse_key_mapping

In [ ]:
xi

current_prediction = {key: round(probability*100, 3) for key , probability in enumerate(neural_network.predict(np.array(xi).reshape(1,-1))[0])}



print(i, network_choice)
print(current_prediction)



In [ ]:
current_prediction.get(5)

In [ ]:
neural_network.predict(np.array(xi).reshape(1,-1))[0]

In [ ]:
neural_network.predict(np.array(xi).reshape(1,-1))